# Market-Basket Analysis

In this notebook we perform Market-Basket analysis by using the Python API **Machine Learning extensions or Mlxtend**.
http://rasbt.github.io/mlxtend/

More specifically, we perform the following two tasks using this API.
- Frequent itemsets generation
- Minining association rules

The API uses the **Apriori** algorithm to generate frequent itemsets.


## Transaction Data Format

The Market-Basket data is typically given using a Python **list of lists** structure.

For the purpose of programmatic processing by the standard APIs (e.g., Mlxtend), the Market-Basket data needs to be stored in a **binary format**. 

More specifically, in a transaction an item can be treated as a binary variable whose value is 1 if the item is present in a transaction and 0 otherwise. This is known as **one-hot encoding**.



## Input Data

The transaction data is stored in a CSV file.

Each row contains a comma separated list of items.

Thus, to create one-hot encoded data we do the following:
- Read from the CSV file as a python **list of lists**. Remove white spaces (empty items).
- One-hot encode the data from list of lists. 


### Create One-Hot Encoded Data
To create a one-hot encoded NumPy boolean array of the transaction data from a Python list of lists, we use the **TransactionEncoder** object from the mlextend API.

The TransactionEncoder uses its "fit" method to learn the unique labels in the dataset, and the "transform" method to transform the input dataset (a Python list of lists) into a one-hot encoded NumPy boolean array.

In [1]:
from time import time

import numpy as np
import pandas as pd
import csv

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Helper Functions for Creating One-Hot Encoded DataFrame Object

We use following functions to create one-hot encoded DataFrame object from the transaction data stored in a CSV file.

- isInList(): Function to determine whether an item/itemset is contained in a list/basket.

- removeEmptyValues(): Function to remove strings (e.g., white space) from the list items.

- generateListOfListsFromCSV(): Function to create a **list of lists** from a CSV file containing transaction data.

- oneHotEncodedDataFrame(): Function to create a one-hot encoded DataFrame object from the Python **list of lists**. 

In [2]:
# Function to determine whether an item/itemset is contained in a list/basket
def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1   

    
# Function to remove strings (e.g., white space) from the list items
def removeEmptyValues(aList, symbolToRemove):
    while(True):
        val = isInList(aList, symbolToRemove)
        if(val == 1):
            aList.remove(symbolToRemove)    
        else:
            break

        

# Function to create a list of lists from a CSV file containing transaction data
def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])
        
    return listOfLists

 

# Function to create a one-hot encoded DataFrame object from the Python list of lists        
def oneHotEncodedDataFrame(data):

    # Transform the transaction data into a one-hot encoded NumPy boolean array 
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)


    # Read the one-hot encoded data as a Pandas DataFrame object
    dataFrame = pd.DataFrame(te_ary, columns=te.columns_)
    
    return dataFrame
        

## Load Data From CSV File and Create One-Hot Encoded DataFrame Object

In [3]:
# Load the CSV file transaction data as a **list of lists**.
dataset = generateListOfListsFromCSV('transactionlarge.csv')

print(len(dataset))

print("\nDataset (list of lists):")
for i in range(len(dataset)):
    print(dataset[i])

# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)

print("\nOne-hot Encoded Dataset:")
print(dataset_onehotencoded)

300

Dataset (list of lists):
['M52', 'M42', 'M34', 'M64', 'M81', 'M43', 'M26', 'M59', 'M16']
['M77', 'M14', 'M15', 'M21', 'M97', 'M82', 'M12']
['M18', 'M93', 'M51', 'M36', 'M49']
['M77', 'M56', 'M14', 'M24', 'M3', 'M95']
['M93', 'M0', 'M95', 'M1', 'M27', 'M2', 'M17']
['M33', 'M6', 'M58', 'M21', 'M89']
['M25', 'M99', 'M72', 'M10', 'M47']
['M0', 'M6', 'M1', 'M21', 'M57', 'M47']
['M74', 'M55', 'M23', 'M28', 'M80', 'M66']
['M98', 'M22', 'M6', 'M13', 'M42', 'M83', 'M92', 'M79', 'M71', 'M84']
['M18', 'M33', 'M4', 'M7', 'M85', 'M34', 'M30', 'M64', 'M59']
['M33', 'M51', 'M24', 'M11', 'M95', 'M79', 'M97', 'M54', 'M10', 'M87']
['M77', 'M18', 'M98', 'M75', 'M70', 'M36', 'M38', 'M57', 'M60']
['M45', 'M35', 'M8', 'M57', 'M80', 'M66']
['M75', 'M14', 'M46', 'M59', 'M10']
['M77', 'M98', 'M75', 'M35', 'M73', 'M22', 'M46', 'M79', 'M29', 'M71', 'M2', 'M81']
['M77', 'M98', 'M51', 'M35', 'M73', 'M79', 'M29', 'M2', 'M30', 'M10']
['M25', 'M45', 'M70', 'M8', 'M41', 'M65', 'M5', 'M15', 'M34', 'M63', 'M96']
['

## Frequent Itemsets Generation

We use Mlxtend API's "apriori" function to generate frequent k-itemsets. This function requires the transaction data to be passed as a one-hot encoded DataFrame object.

        apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0, low_memory=False)


### Parameters

- df : pandas DataFrame or pandas SparseDataFrame

        pandas DataFrame the encoded format. The allowed values are either 0/1 or True/False. 

- min_support : float (default: 0.5)

        A float between 0 and 1 for minumum support of the itemsets returned. The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.

- use_colnames : bool (default: False)

        If True, uses the DataFrames' column names in the returned DataFrame instead of column indices.

- max_len : int (default: None)

        Maximum length of the itemsets generated. If None (default) all possible itemsets lengths (under the apriori condition) are evaluated.

- verbose : int (default: 0)

        Shows the number of iterations if >= 1 and low_memory is True. If=1 and low_memory is False, shows the number of combinations.

- low_memory : bool (default: False)

        If True, uses an iterator to search for combinations above min_support. Note that while low_memory=True should only be used for large dataset if memory resources are limited, because this implementation is approx. 3-6x slower than the default.


- Returns

pandas DataFrame with columns ['support', 'itemsets'] of all itemsets that are >= min_support and < than max_len (if max_len is not None). 

Each itemset in the 'itemsets' column is of type frozenset, which is a Python built-in type that behaves similarly to sets except that it is immutable (For more info, see https://docs.python.org/3.6/library/stdtypes.html#frozenset).

## Generate All Frequent Itemsets with at least 30% Support

In [16]:
t0 = time()   
ms= 5/300
frequent_itemsets = apriori(dataset_onehotencoded, min_support=ms, use_colnames=True)

time_apriori = time() - t0

print("Frequent Itemsets Generation: Processing Time = %0.3fs\n" % time_apriori)

print(frequent_itemsets)

Frequent Itemsets Generation: Processing Time = 0.215s

      support                   itemsets
0    0.073333                       (M0)
1    0.093333                       (M1)
2    0.116667                      (M10)
3    0.093333                      (M11)
4    0.076667                      (M12)
..        ...                        ...
931  0.016667  (M29, M38, M45, M93, M62)
932  0.016667   (M3, M87, M66, M92, M68)
933  0.016667  (M53, M63, M87, M72, M35)
934  0.016667  (M84, M89, M73, M45, M44)
935  0.016667  (M50, M60, M56, M92, M68)

[936 rows x 2 columns]


In [10]:
#733, 259, 185
t0 = time()   
ms= 3/300
print(ms)
frequent_itemsets = apriori(dataset_onehotencoded, min_support=ms, use_colnames=True)

time_apriori = time() - t0

print("Frequent Itemsets Generation: Processing Time = %0.3fs\n" % time_apriori)

print(len(frequent_itemsets))

0.01
Frequent Itemsets Generation: Processing Time = 0.719s

3001


## Frequent Itemsets: Selecting and Filtering Results

The above code generates ALL frequent itemsets.

We can select only the desired itemsets of a **specific length** as well (e.g., frequent 2-itemsets).

To do this we add a new column to the "frequent_itemsets" DataFrame that stores the length of each itemset.

In [17]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

print(frequent_itemsets)

      support                   itemsets  length
0    0.073333                       (M0)       1
1    0.093333                       (M1)       1
2    0.116667                      (M10)       1
3    0.093333                      (M11)       1
4    0.076667                      (M12)       1
..        ...                        ...     ...
931  0.016667  (M29, M38, M45, M93, M62)       5
932  0.016667   (M3, M87, M66, M92, M68)       5
933  0.016667  (M53, M63, M87, M72, M35)       5
934  0.016667  (M84, M89, M73, M45, M44)       5
935  0.016667  (M50, M60, M56, M92, M68)       5

[936 rows x 3 columns]


## Frequent Itemsets: Select Itemsets Based on Criteria 

Now we can select an itemset based on criteria (such as length, support).

In [18]:
# Select all frequent 3-itemsets
frequent_3_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 3)]

print("\nFrequent 3-itemsets:")
print(frequent_3_itemsets)


# Select all frequent 3-itemsets with minimum support 0.5
frequent_3_itemsets = frequent_itemsets[ (frequent_itemsets['length'] == 3) &
                                        (frequent_itemsets['support'] >= 0.01) ]

print("\nFrequent 3-itemsets with Minimum Support 0.01:")
print(frequent_3_itemsets)


Frequent 3-itemsets:
      support         itemsets  length
643  0.016667   (M0, M23, M55)       3
644  0.016667   (M0, M47, M51)       3
645  0.016667   (M0, M72, M47)       3
646  0.020000   (M0, M74, M47)       3
647  0.016667   (M0, M72, M51)       3
..        ...              ...     ...
823  0.023333  (M66, M92, M68)       3
824  0.016667  (M87, M92, M66)       3
825  0.016667  (M87, M92, M68)       3
826  0.020000  (M89, M84, M73)       3
827  0.016667  (M79, M98, M77)       3

[185 rows x 3 columns]

Frequent 3-itemsets with Minimum Support 0.01:
      support         itemsets  length
643  0.016667   (M0, M23, M55)       3
644  0.016667   (M0, M47, M51)       3
645  0.016667   (M0, M72, M47)       3
646  0.020000   (M0, M74, M47)       3
647  0.016667   (M0, M72, M51)       3
..        ...              ...     ...
823  0.023333  (M66, M92, M68)       3
824  0.016667  (M87, M92, M66)       3
825  0.016667  (M87, M92, M68)       3
826  0.020000  (M89, M84, M73)       3
827  0.01

## Association Rules

We use Mlxtend API's "association_rules" function to generate association rules (as a DataFrame object) including the metrics 'score', 'confidence', and 'lift'. This function requires the transaction data to be passed as a one-hot encoded DataFrame object.

        association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)



### Parameters

- df : pandas DataFrame

        pandas DataFrame of frequent itemsets with columns ['support', 'itemsets']

- metric : string (default: 'confidence')

        Metric to evaluate if a rule is of interest. Automatically set to 'support' if support_only=True. Otherwise, supported metrics are 'support', 'confidence', 'lift', 'leverage', and 'conviction.


In [41]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

#rules.head() # use this method if the number of rules is very large
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(M1),(M0),0.093333,0.073333,0.01,0.107143,1.461039,0.003156,1.037867
1,(M0),(M1),0.073333,0.093333,0.01,0.136364,1.461039,0.003156,1.049825
2,(M0),(M17),0.073333,0.090000,0.01,0.136364,1.515152,0.003400,1.053684
3,(M17),(M0),0.090000,0.073333,0.01,0.111111,1.515152,0.003400,1.042500
4,(M2),(M0),0.136667,0.073333,0.01,0.073171,0.997783,-0.000022,0.999825
...,...,...,...,...,...,...,...,...,...
14063,(M93),"(M73, M38, M45, M62, M29)",0.106667,0.010000,0.01,0.093750,9.375000,0.008933,1.092414
14064,(M38),"(M73, M93, M45, M62, M29)",0.103333,0.010000,0.01,0.096774,9.677419,0.008967,1.096071
14065,(M45),"(M73, M93, M38, M62, M29)",0.113333,0.010000,0.01,0.088235,8.823529,0.008867,1.085806
14066,(M62),"(M73, M93, M38, M45, M29)",0.090000,0.010000,0.01,0.111111,11.111111,0.009100,1.113750


## Association Rules: Selecting Results

We can select association rules based on criteria (antecedent length, confidece, lift, etc.).

To do this we add a new column to the "rules" DataFrame that stores the length of the antecedents.

In [8]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

#rules.head() # use this method if the number of rules is very large
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(M1),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125,1
1,(M2),(M1),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250,1
2,(M1),(M3),0.750,0.375,0.375,0.500000,1.333333,0.09375,1.250,1
3,(M3),(M1),0.375,0.750,0.375,1.000000,1.333333,0.09375,inf,1
4,(M1),(M4),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750,1
5,(M4),(M1),0.750,0.750,0.500,0.666667,0.888889,-0.06250,0.750,1
6,(M4),(M2),0.750,0.625,0.500,0.666667,1.066667,0.03125,1.125,1
7,(M2),(M4),0.625,0.750,0.500,0.800000,1.066667,0.03125,1.250,1
8,(M5),(M2),0.500,0.625,0.375,0.750000,1.200000,0.06250,1.500,1
9,(M2),(M5),0.625,0.500,0.375,0.600000,1.200000,0.06250,1.250,1


## Association Rules: Select Association Rules Based on Criteria

Following example select association rules based on 3 criteria.
- Antecedent length
- Confidence
- Lift

In [9]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.65) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
19,"(M2, M4)",(M5),0.500,0.50,0.375,0.75,1.500000,0.12500,2.0,2
20,"(M5, M2)",(M4),0.375,0.75,0.375,1.00,1.333333,0.09375,inf,2
